In [1]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 4.2 MB/s 
     |████████████████████████████████| 16.7 MB 49.4 MB/s 
     |████████████████████████████████| 6.3 MB 8.3 MB/s 


In [2]:
import geopandas as gpd
import pandas as pd
import numpy as num
from shapely.geometry import Point

In [7]:
raw_data = gpd.read_file("nv_vest_20.shp")
census_data = pd.read_excel('2020PL94-171_ADJPOP11-13-2021_Precincts.xlsx') 
points = raw_data.copy()
census = census_data.copy()

In [8]:
#points['geometry'] = points['geometry'].centroid
# geoid is separated to STATEFP, COUNTYFP, VTDST --> Integrate them for GEOID

points = points.assign(GEOID20 = points.STATEFP.astype(str) + '' + points.COUNTYFP.astype(str) + '' + points.VTDST.astype(str))
points = pd.DataFrame(points.drop(columns=['STATEFP', 'COUNTYFP', 'VTDST']))
points.rename(columns={'G20PREDBID': 'DemocraticVote', 'G20PRERTRU': 'RepublicanVote', 
                       'G20PRELJOR': 'LibertarianVote', 'G20PREIBLA': 'IndependentVote', 
                       'G20PREONON': 'OtherPartiesVote'}, inplace=True)
points.head()

,COUNTY,NAME,DemocraticVote,RepublicanVote,LibertarianVote,IndependentVote,OtherPartiesVote,geometry,GEOID20
0,Churchill,CHURCHILL PRECINCT 14,100,500,12,1,8,"POLYGON Z ((-119.06227 39.34643 0.00000, -119....",32001000014
1,Churchill,CHURCHILL PRECINCT 19,117,504,13,1,6,"POLYGON Z ((-119.11763 39.50918 0.00000, -119....",32001000019
2,Esmeralda,ESMERALDA GOLDFIELD PRECINCT 1,36,179,4,1,3,"POLYGON Z ((-117.43051 37.78696 0.00000, -117....",32009000001
3,Esmeralda,ESMERALDA SILVER PEAK PRECINCT 2,4,42,0,0,1,"POLYGON Z ((-117.87888 37.91707 0.00000, -117....",32009000002
4,Esmeralda,ESMERALDA FISH LAKE VALLEY PRECINCT 3,30,165,2,0,2,"POLYGON Z ((-118.42300 37.89637 0.00000, -118....",32009000003


In [9]:
# NHS = Not Hispanic or Latino
# AIA = American Indian or Alaska Native
# ASN = Asian
# BLK = African American
# NHP = Native Hawaiian or Other Pacific Islander Alone

census.rename(columns={'NAME20': 'NAME', 'ADJPOP':'Total', 'TAWHITEALN':'White',
                       'TABLACKCMB':'Black', 'TAAIANCMB':'AmericanIndian', 'TAASIANCMB':'Asian',
                       'TANHOPICMB':'NativeHawaiian', 'TAOTHERALN':'OtherRace',
                       'TA2RACE': 'TwoRace', 'TAHISPANIC':'Hispanic'}, inplace=True)

census = pd.DataFrame(census.drop(columns=['STATEFP20', 'COUNTYFP20', 'VTDST20', 'GEOID']))
census.head()

,GEOID20,NAME,Total,White,Black,AmericanIndian,Asian,NativeHawaiian,OtherRace,TwoRace,Hispanic
0,32001000001,CHURCHILL PRECINCT 01,1743,1280,93,105,125,9,71,202,235
1,32001000002,CHURCHILL PRECINCT 02,2063,1513,142,123,93,28,92,208,266
2,32001000003,CHURCHILL PRECINCT 03,1185,823,68,74,102,9,57,130,195
3,32001000004,CHURCHILL PRECINCT 04,1768,1261,108,119,104,29,109,215,253
4,32001000005,CHURCHILL PRECINCT 05,1477,977,67,125,116,15,95,200,265


In [10]:
merge_table = census.merge(points, on='GEOID20')
merge_table = pd.DataFrame(merge_table.drop(columns=['NAME_y', 'COUNTY']))
merge_table.rename(columns={'NAME_x': 'NAME'}, inplace=True)
merge_table.head()


#Save merge_table as json file
#gdf = gpd.GeoDataFrame(merge_table[['GEOID20', 'NAME', 'Total', 'White', 'Black', 'AmericanIndian', 'Asian', 'NativeHawaiian', 
#                                    'OtherRace', 'TwoRace', 'Hispanic', 'DemocraticVote', 'RepublicanVote',
#                                    'LibertarianVote', 'IndependentVote', 'OtherPartiesVote', 'geometry']])
#gdf.to_file('D:/Preprocessing/nevada_census.json', driver='GeoJSON')

,GEOID20,NAME,Total,White,Black,AmericanIndian,Asian,NativeHawaiian,OtherRace,TwoRace,Hispanic,DemocraticVote,RepublicanVote,LibertarianVote,IndependentVote,OtherPartiesVote,geometry
0,32001000001,CHURCHILL PRECINCT 01,1743,1280,93,105,125,9,71,202,235,227,510,14,3,9,"POLYGON Z ((-118.77683 39.48451 0.00000, -118...."
1,32001000002,CHURCHILL PRECINCT 02,2063,1513,142,123,93,28,92,208,266,267,498,19,1,9,"POLYGON Z ((-118.76777 39.47492 0.00000, -118...."
2,32001000003,CHURCHILL PRECINCT 03,1185,823,68,74,102,9,57,130,195,151,352,26,1,10,"POLYGON Z ((-118.78283 39.46955 0.00000, -118...."
3,32001000004,CHURCHILL PRECINCT 04,1768,1261,108,119,104,29,109,215,253,216,383,16,4,11,"POLYGON Z ((-118.77245 39.47138 0.00000, -118...."
4,32001000005,CHURCHILL PRECINCT 05,1477,977,67,125,116,15,95,200,265,142,365,21,3,10,"POLYGON Z ((-118.77236 39.46207 0.00000, -118...."


In [18]:
geo = gpd.GeoDataFrame(merge_table)
edges = []
for i, precinct1 in geo.iterrows():
  for j, precinct2 in geo.iterrows():
    if j <= i:
      continue
    if precinct1.geometry.overlaps(precinct2.geometry.buffer(0.009)):
      edges.append((i, j))
      edges.append((j, i))
  print(i)
print(edges)

0
1
2
3
4
5
6
7
8
9
10


KeyboardInterrupt: ignored